# Caculation of mean squared displacement from Video particle tracking
This program analyzes the trajectories obtain from video microscopy of beads immersed in a medium. The tracking of the beads is done through trackmate plugin of FIJI-ImageJ and the trajectories are saved as CSV files from the spot statistics window in Trackmate. This program can open the trajectories and perform drift correction, calcualtes the MSD of each particle, calculate the ensemble MSD, fits the MSD to extract the diffusion coefficient and calcualtes and writes the viscosity of the medium.  

This program is published as a part of associated code for the manuscript "*Programmable Viscoelasticity in Protein-RNA Condensates with Disordered Sticker-Spacer Polypeptides*" by Alshareedah and coworkers. See also 
* *Alshareedah, I., Thurston, G. M. & Banerjee, P. R. Quantifying Viscosity and Surface Tension of Multi-Component Protein-Nucleic Acid Condensates. Biophysical journal, doi:https://doi.org/10.1016/j.bpj.2021.01.005 (2021).* 

The function ReadTable() is adapted from the microrheologypy software https://launchpad.net/microrheologypy/ and updated to Python 3 and modefied to read CSV files produced by trackmate. See *Maier, T., & Haraszti, T. (2012). Python algorithms in particle tracking microrheology. Chemistry Central Journal, 6(1), 1-9*.


Program author: Ibraheem Alshareedah. 
Last updated: Oct 6th 2021

In [ ]:

import matplotlib.pyplot as plt
import numpy as nu
from lumicks import pylake
import math
from multipletau import autocorrelate
import array
import lmfit
import scipy as sp
import numpy as np
from os import listdir
import sys, os.path


In [ ]:
# this function is an update and modefied version of ReadTable() in microrheologypy software https://launchpad.net/microrheologypy/
def ReadTable(filename, beadindx = -1, X=0, Y=1, indx=2, skip=1):

    if os.path.isfile(filename):
        fp = open(filename, 'rt')
    else:
        print("file not found: %s" %filename) 
        return None
    
    a = fp.readlines()
    fp.close()

   
    a = a[skip:]
    Ni = len(a)
    
    t = (a[0].strip()).split(',')
    Nj = len(t)-1

    
    res = []
    for i in range(Ni):
        t = a[i].strip()
        
        l = []
        t = t.split(',')

        for j in range(Nj):
            l.append(float(t[j+1]))

        res.append(l)
    
    print("Found: %d lines" %len(res))

  
    res = nu.asarray(res)
    #how many beads? beadindx is the column holding the counter:
    Nb = nu.unique(res[:,beadindx-1])
    print("Max: %d beads" %len(Nb)) 

    poslist = []
    for i in Nb:

        #bindx highlights those lines containing bead i:
        bindx = (res[:,beadindx-1] == i)
        #poslist is a list of dicts
        #each dict holds 'X', 'Y' for position arrays, 
        #and 'indx' for the image indices (may not be continuous)
        poslist.append({"X":res[bindx,X-1], "Y":res[bindx,Y-1],\
                        "indx":res[bindx,indx-1]})

    print("Found: %d beads" %len(poslist))

    return poslist


In [ ]:
import os
# enter directory where the csv files from trackmate are saved
testdir = r"E:\"
os.listdir(testdir)
os.chdir(testdir)
os.getcwd()
#enter filename
fname='RG5-T40 200 nm beads 100x 10 ms expo 10 ms interval_1'
pos=ReadTable(fname+'.csv',beadindx = 2, X=4, Y=5, indx=7, skip=1)

### Calculate Center of Mass for drift correction

In [ ]:
#calculate center of mass


sizes = np.zeros((len(pos),))
for i in range(len(pos)): 
    sizes[i]=len(pos[i]['X'])
lastframe=min(sizes)-1
XCOM=np.zeros((int(lastframe),))
YCOM=np.zeros((int(lastframe),))
for i in range(int(lastframe)):
    Xavg=0
    Yavg=0
    for j in range(len(pos)):
        Xavg=Xavg+pos[j]['X'][i]
        Yavg=Yavg+pos[j]['Y'][i]
    XCOM[i]=Xavg/len(pos)
    YCOM[i]=Yavg/len(pos)
plt.plot(XCOM)


### calculate individual MSDs

In [ ]:
# calculate MSDs
frametime=0.01 #in seconds
msds = []
xpos=[]
ypos=[]
i=[]
msdsizes=[]
for l in range(len(pos)):
    ind=pos[l]['indx'][0:int(lastframe)]
    xpos=pos[l]['X'][0:int(lastframe)]-XCOM
    ypos=pos[l]['Y'][0:int(lastframe)]-YCOM
    a,b=np.polyfit(ind,xpos,1)
    error = a*ind+b
    #xpos = xpos-error 
    a,b=np.polyfit(ind,ypos,1)
    error = a*ind+b
    #ypos = ypos-error 
    plt.plot(xpos,ypos)
    nData = len(xpos)
    numberOfdeltaT = int(nData/4)
    msd = np.zeros((numberOfdeltaT,4))
    

    k=0
    for dt in range(numberOfdeltaT):
        if dt==0:
            continue
        deltax = xpos[1+dt:len(xpos)-1]-xpos[1:len(xpos)-1-dt]
        deltay = ypos[1+dt:len(ypos)-1]-ypos[1:len(ypos)-1-dt]

        sD = deltax**2+deltay**2
        msd[k,0]  =frametime*dt
        msd[k,1]  =np.mean(sD)
        msd[k,2]  =np.std(sD)
        msd[k,3]  =len(sD)
        k=k+1
    msds.append({"time":msd[:,0], "MSD":msd[:,1],"DMSD":msd[:,2], "MSDsize":len(msd)})
    msdsizes.append(len(msd))
   
    xpos=[]
    ypos=[]
    i=[]

    plt.xlabel('X (um)')
    plt.ylabel('Y (um)')
    plt.savefig(fname+'tracks.png')

# Compute average MSD

In [ ]:
#compute average MSD
lastind=np.min(msdsizes)-1
totmsd =np.zeros((lastind,))
for i in range(len(msds)):
    mmd = np.array(msds[i]['MSD'])
    time= msds[i]['time']
    totmsd=totmsd+mmd[0:lastind]
msdavg= totmsd/len(msds)
time=time[0:lastind]
plt.loglog(time,msdavg,'o')
np.savetxt(fname+'MSD.txt',np.transpose([time,msdavg]), delimiter='\t')   # X is an array

# Perform MSD Fit and calculate the viscosity

In [ ]:
#Perform the fitting 

def MSDfitfit(t, D,alpha,N):
    return 4*D*t**alpha + N

modely = lmfit.Model(MSDfitfit)
params = modely.make_params(D=1.,alpha=1.,N=0)


params['D'].set(value=.2, vary=True)
params['alpha'].set(value=1.0, vary=True)
params['N'].set(value=0.,max=0.00003,min=0., vary=True)
start=0
end=100

fitresy = modely.fit(msdavg[start:end], t=time[start:end], params=params) 
                 #  method='least_squares')
print(fitresy.fit_report())

plt.loglog(time[start:end],msdavg[start:end],'o')
plt.loglog(time[start:end], fitresy.best_fit)
plt.loglog(time[start:end], fitresy.init_fit)
plt.xlabel(r'$\tau (s)$')
plt.ylabel(r'$MSD(\tau)$')
plt.savefig(fname+'MSDfit.png',bbox_inches='tight')
timenew=np.logspace(-5,1,1000000)
#Jnew=modely.eval(fitresy.params,t=timenew)
np.savetxt(fname+'MSDandfit.txt',np.transpose([time[start:end],msdavg[start:end],fitresy.best_fit]))

In [ ]:
#calculate viscostiy
R=0.1 #in microns
eta = 0.0002165/(R*fitresy.best_values['D'])
eta

In [ ]:
#save fitting report and viscosity
import csv
f = open(fname+'_fitting_report.txt','w')
f.write(fitresy.fit_report())
f.close()

out=open('viscosity.txt','a')
out.write(str(eta)+'\n')
out.close()